In [15]:
# --- Celda 1: Setup y Definición de Países ---
!pip install papermill pandas 
print("Dependencias instaladas.")

import papermill as pm
import pandas as pd
import os

# Definir la URL del dataset
URL_COVID_DATA = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
NOMBRE_PLANTILLA = 'notebook-ariel.ipynb' 
ARCHIVOS_RESULTADOS = [] 

# ----------------------------------------------------
# LÓGICA DE SELECCIÓN DE LOS PRIMEROS 5 PAÍSES
# ----------------------------------------------------

REGIONES_DE_PRUEBA = ["Argentina", "Brazil", "Chile", "US", "Spain", "Germany", "France"] # Fallback

try:
    df_paises = pd.read_csv(URL_COVID_DATA)
    # 1. Obtiene la lista completa de todos los países
    REGIONES_TODAS = df_paises['Country/Region'].unique().tolist()
    
    # 2. Selecciona SOLO los primeros 5 países
    REGIONES_A_ANALIZAR = REGIONES_TODAS[:5] 
    
    print("✅ Lista de regiones obtenida de la fuente de datos.")
    print(f"⏩ Se seleccionaron automáticamente los primeros 5 países: {REGIONES_A_ANALIZAR}")

except Exception as e:
    print(f"❌ ERROR al cargar el CSV de la URL. Usando lista de prueba.")
    REGIONES_A_ANALIZAR = REGIONES_DE_PRUEBA[:5] # Usa los 5 primeros del fallback
    
# ----------------------------------------------------

# Crear el directorio de salida (output)
if not os.path.exists('output'):
    os.makedirs('output')
    print("Directorio 'output/' creado.")
    
print(f"\nIniciando pipeline para {len(REGIONES_A_ANALIZAR)} regiones.")

Dependencias instaladas.
✅ Lista de regiones obtenida de la fuente de datos.
⏩ Se seleccionaron automáticamente los primeros 5 países: ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola']

Iniciando pipeline para 5 regiones.


In [16]:
# --- Celda 2: Orquestación y Ejecución Parametrizada (Papermill) ---

# 1. Verificación de existencia de la plantilla ANTES de empezar el bucle
NOMBRE_PLANTILLA = 'notebook-ariel.ipynb' 
if not os.path.exists(NOMBRE_PLANTILLA):
    print(f"❌ ERROR CRÍTICO: La plantilla '{NOMBRE_PLANTILLA}' no se encontró en la ruta actual.")
    print("El pipeline no puede continuar. Asegúrate de que el archivo esté guardado.")
else:
    print(f"✅ Plantilla '{NOMBRE_PLANTILLA}' encontrada. Iniciando ejecución...")

    # Usamos la lista de REGIONES_A_ANALIZAR definida en la Celda 1
    
    for region in REGIONES_A_ANALIZAR:
        
        # Rutas de Archivos (usando el subdirectorio 'output')
        region_safe = region.replace(' ', '_')
        nombre_output_nb = f"output/informe_limpieza_{region_safe}.ipynb"
        nombre_csv_salida = f"output/indicadores_{region_safe}.csv"
        
        # 2. Agregar la ruta a la lista ANTES de ejecutar (para seguimiento)
        ARCHIVOS_RESULTADOS.append(nombre_csv_salida)
        
        print(f"\n⚙️ Ejecutando análisis para: {region}...")
        
        # 3. Llamada a Papermill con manejo de errores
        try:
            pm.execute_notebook(
                input_path=NOMBRE_PLANTILLA,
                output_path=nombre_output_nb,
                parameters=dict(
                    REGION_PARAMETRO=region,
                    NOMBRE_ARCHIVO_SALIDA=nombre_csv_salida
                )
            )
            # Si tiene éxito, Papermill crea el .ipynb y el código interno de la plantilla crea el .csv
            print(f"✅ Ejecución exitosa para {region}. CSV esperado en {nombre_csv_salida}")

        except Exception as e:
            print(f"❌ Falló la ejecución de Papermill para {region}. Error: {e}")
            # Si falla, ELIMINAMOS el último archivo CSV agregado de la lista para que la compilación final no falle.
            if ARCHIVOS_RESULTADOS:
                 ARCHIVOS_RESULTADOS.pop()
            print("⚠️ El CSV fallido fue removido de la lista de compilación final.")


    print("\n🎉 ¡Ejecución de todas las plantillas completada!")

Passed unknown parameter: REGION_PARAMETRO


Passed unknown parameter: NOMBRE_ARCHIVO_SALIDA
Input notebook does not contain a cell with tag 'parameters'


✅ Plantilla 'notebook-ariel.ipynb' encontrada. Iniciando ejecución...

⚙️ Ejecutando análisis para: Afghanistan...


Executing:   0%|          | 0/5 [00:00<?, ?cell/s]

Passed unknown parameter: REGION_PARAMETRO
Passed unknown parameter: NOMBRE_ARCHIVO_SALIDA
Input notebook does not contain a cell with tag 'parameters'


...

Executing:   0%|          | 0/5 [00:00<?, ?cell/s]

Passed unknown parameter: REGION_PARAMETRO
Passed unknown parameter: NOMBRE_ARCHIVO_SALIDA
Input notebook does not contain a cell with tag 'parameters'


✅ Ejecución exitosa para Andorra. CSV esperado en output/indicadores_Andorra.csv

⚙️ Ejecutando análisis para: Angola...


Executing:   0%|          | 0/5 [00:00<?, ?cell/s]

✅ Ejecución exitosa para Angola. CSV esperado en output/indicadores_Angola.csv

🎉 ¡Ejecución de todas las plantillas completada!


In [17]:
# --- Celda 3: Compilación del Reporte Final Consolidado ---

list_df = []
for file in ARCHIVOS_RESULTADOS:
    if os.path.exists(file):
        df_temp = pd.read_csv(file)
        list_df.append(df_temp)
    # No mostramos error si el archivo no existe, ya que Papermill debe asegurarlo

# Concatenar (unir) todos los DataFrames
if list_df:
    df_reporte_final = pd.concat(list_df, ignore_index=True)
    
    # Guardar el Reporte Final Consolidado
    NOMBRE_REPORTE_FINAL = "REPORTE_FINAL_INDICADORES_CONSOLIDADO.csv"
    df_reporte_final.to_csv(NOMBRE_REPORTE_FINAL, index=False)

    print(f"\n✅ REPORTE FINAL CONSOLIDADO GENERADO: {NOMBRE_REPORTE_FINAL}")
    
    # Mostrar el resultado final
    print("\nRESUMEN CONSOLIDADO DE INDICADORES Y LIMPIEZA:")
    display(df_reporte_final)
else:
    print("\n❌ No se pudo compilar el reporte final. Verifique que 'output/' contenga los archivos CSV.")


❌ No se pudo compilar el reporte final. Verifique que 'output/' contenga los archivos CSV.
